In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
! mkdir ~/.kaggleb

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jhoward/lsun_bedroom

100% 8.89G/8.89G [05:01<00:00, 42.8MB/s]
100% 8.89G/8.89G [05:01<00:00, 31.7MB/s]


In [ ]:
!unzip lsun_bedroom.zip

In [1]:
!git clone https://github.com/mikolajzalewski/Deep_Learning.git

Cloning into 'Deep_Learning'...
remote: Enumerating objects: 51101, done.
remote: Counting objects: 100% (1091/1091), done.
remote: Compressing objects: 100% (486/486), done.
remote: Total 51101 (delta 607), reused 986 (delta 585), pack-reused 50010
Receiving objects: 100% (51101/51101), 768.10 MiB | 27.28 MiB/s, done.
Resolving deltas: 100% (608/608), done.
Updating files: 100% (128/128), done.


In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [25]:
%cd /content/gdrive/MyDrive/Colab Notebooks/data0/lsun/bedroom

/content/gdrive/MyDrive/Colab Notebooks/data0/lsun/bedroom


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import os
import cv2
import numpy as np
import pandas as pd

from torchvision.models import inception_v3
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from gan_package.gan import GAN
from gan_package.vanillaGAN import VanillaGAN_Generator, VanillaGAN_Discriminator
from gan_package.dcGAN import DCGAN_Generator, DCGAN_Discriminator

In [2]:
image_size = 256
batch_size = 128
workers = 2
root = 'lsun/bedroom/0/0/'

In [3]:
dataset = dset.ImageFolder(root=root,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

In [ ]:
batch_sizes = [16, 32, 64, 128, 256]
learning_rate_gen = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]
learning_rate_disc = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]

inception_model = inception_v3(pretrained=True, transform_input=False, aux_logits=True)

In [ ]:
results = pd.DataFrame(columns=['batch_size', 'learning_rate', 'fid_score', 'loss_discriminator', 'loss_generator'])


for batch_size in batch_sizes:
    for learning_rate in learning_rate_gen:
            print('Learning_rate:', learning_rate, 'Batch_size:', batch_size)
            dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)
            
            img_shape= dataloader.dataset[0][0].shape
            n_out = torch.prod(torch.tensor(img_shape))
            latent_dim = 100

            vanilla_generator = VanillaGAN_Generator(latent_dim=latent_dim, img_shape=img_shape, n_out=n_out)
            vanilla_discriminator = VanillaGAN_Discriminator(img_shape=img_shape)
            vanilla_gan = GAN(generator=vanilla_generator, discriminator=vanilla_discriminator, inception_model=inception_model)

            criterion = nn.BCELoss()

            vanilla_generator_optimizer = optim.Adam(vanilla_generator.parameters(), lr=learning_rate)
            vanilla_discriminator_optimizer = optim.Adam(vanilla_discriminator.parameters(), lr=learning_rate)

            vanilla_gan.train(dataloader=dataloader,
                  discriminator_optimizer=vanilla_discriminator_optimizer,
                  generator_optimizer=vanilla_generator_optimizer,
                  criterion=criterion,
                  num_epochs=5)
            
            results.append({'batch_size': batch_size, 'learning_rate': learning_rate, 'fid_score': vanilla_gan.history['fid_scores'][-1], 'loss_discriminator': vanilla_gan.history['d_losses'][-1], 'loss_generator': vanilla_gan.history['d_losses'][-1]}, ignore_index=True)